In [ ]:
import networkx as nx
from torch_geometric.utils import from_networkx
import os
import sys

# gat → models → src
src_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
if src_path not in sys.path:
    sys.path.append(src_path)


import import_ipynb 
from utils.networkx_to_pyg_wrapper import from_networkx_with_edge_attrs



In [18]:
data_list = []
for i in range(12):  # 0 bis 11
    path = f"../../graphs/2023/bike_network_2023_{i}.graphml"
    G_nx = nx.read_graphml(path)
    G_nx = nx.DiGraph(G_nx)
    data = from_networkx(G_nx, edge_attrs=['tracks', 'month', 'year', 'speed_rel'])
    data_list.append(data)


TypeError: from_networkx() got an unexpected keyword argument 'edge_attrs'

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATv2Conv

class GATv2(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, edge_dim, heads=1):
        super(GATv2, self).__init__()

        # First GATv2 layer, with edge attributes
        self.gat1 = GATv2Conv(in_channels, hidden_channels, heads=heads, edge_dim=edge_dim)

        # Second GATv2 layer, output dimension = out_channels
        self.gat2 = GATv2Conv(hidden_channels * heads, out_channels, heads=1, edge_dim=edge_dim)

    def forward(self, x, edge_index, edge_attr):
        # Apply first GATv2 layer with edge attributes
        x = self.gat1(x, edge_index, edge_attr)
        x = F.elu(x)

        # Apply second GATv2 layer
        x = self.gat2(x, edge_index, edge_attr)
        return x
